In [1]:
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
NOTEBOOKS_DIRECTORY = os.getcwd()
DATA_DIRECTORY = os.path.join(os.path.dirname(NOTEBOOKS_DIRECTORY), 'data')

data_files = [file for file in os.listdir(DATA_DIRECTORY) if '.csv' in file.lower()]
data_files = sorted(data_files, reverse = True)
portfolio_file = os.path.join(DATA_DIRECTORY, data_files[0])
portfolio_df = pd.read_csv(portfolio_file, skiprows=3)

portfolio_df = portfolio_df[['Symbol', 'Description', 'Quantity', 'Cost Basis', '% Of Account']]
portfolio_df = portfolio_df.loc[~portfolio_df['Symbol'].isin(['Cash & Cash Investments', 'Account Total'])]
portfolio_df = portfolio_df.replace({'BRK/B':'BRK-B'})

print(portfolio_df.shape)
print(portfolio_df.info())

portfolio_df.head(5)

In [ ]:
def get_profile_data(portfolio, attribute):
    """
    Company Profile of Items
    """
    
    stock = portfolio['Symbol']
    profile = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{stock}')
    profile = profile.json()
    
    try:
        return profile['profile'][attribute]
    except:
        return None

def get_quote_data(portfolio, attribute):
    """
    Company Quote Group of Items
    """
    
    stock = portfolio['Symbol']
    company_quote = requests.get(f'https://financialmodelingprep.com/api/v3/quote/{stock}')
    company_quote = company_quote.json()
    
    try:
        return float('{0:.2f}'.format(company_quote[0][attribute]))
    except:
        return None

def get_balance_sheet_data(portfolio, attribute, quarter):
    """
    Balance Sheet Group of Items
    """
    
    stock = portfolio['Symbol']
    bs = requests.get(f"https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{stock}?period=quarter")
    bs = bs.json()
    
    try:
        return bs['financials'][quarter][attribute]
    except:
        return None

def get_income_statement_data(portfolio, attribute, quarter):
    """
    Income Statement Group of Items
    """
    
    stock = portfolio['Symbol']
    income_stat = requests.get(f"https://financialmodelingprep.com/api/v3/financials/income-statement/{stock}?period=quarter")
    income_stat = income_stat.json()
    
    try:
        return income_stat['financials'][quarter][attribute]
    except:
        return None

In [ ]:
portfolio_df['price'] = portfolio_df.apply(lambda stock: get_quote_data(stock, 'price'), axis=1)

In [ ]:
portfolio_df